In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
import joblib
from scipy.stats import uniform, randint
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance

#  Régression Linéaire avec ACP 

In [ ]:
# --- Chargement des jeux d'entraînement et de test ---
train = pd.read_csv("train_data_with_genres.csv", sep=";", encoding="utf-8", low_memory=False)
test = pd.read_csv("test_data_with_genres.csv", sep=";", encoding="utf-8", low_memory=False)

# --- Normalisation des décimales : remplace ',' par '.' (format EU -> US) ---
train = train.replace(",", ".", regex=True)
test = test.replace(",", ".", regex=True)

# --- Séparation de la cible et suppression de 'popularity' des features ---
y = pd.to_numeric(train["popularity"], errors="coerce").fillna(0)
train = train.drop(columns=["popularity"], errors="ignore")

# --- Sauvegarde des identifiants test et retrait de 'row_id' des features ---
test_ids = test["row_id"]
train = train.drop(columns=["row_id"], errors="ignore")
test = test.drop(columns=["row_id"], errors="ignore")

# --- Liste des variables audio pour l'ACP ---
audio_features = [
    "danceability", "energy", "loudness", "speechiness", "acousticness",
    "instrumentalness", "liveness", "valence", "tempo"
]

# --- Séparation des blocs : audio (pour ACP) vs autres variables (métadonnées, one-hot, etc.) ---
X_audio = train[audio_features].apply(pd.to_numeric, errors="coerce").fillna(0)
X_other = train.drop(columns=audio_features).apply(pd.to_numeric, errors="coerce").fillna(0)

test_audio = test[audio_features].apply(pd.to_numeric, errors="coerce").fillna(0)
test_other = test.drop(columns=audio_features).apply(pd.to_numeric, errors="coerce").fillna(0)

# --- Sécurisation : aligne les colonnes non-audio du test sur celles du train ---
missing_cols = set(X_other.columns) - set(test_other.columns)
for c in missing_cols:
    test_other[c] = 0
test_other = test_other[X_other.columns]

# --- Standardisation des features audio (centrage-réduction) avant ACP ---
scaler = StandardScaler()
X_audio_scaled = scaler.fit_transform(X_audio)
test_audio_scaled = scaler.transform(test_audio)

# --- ACP : conserve 90% de la variance sur le bloc audio ---
pca = PCA(n_components=0.90, svd_solver='full')
X_pca = pca.fit_transform(X_audio_scaled)
test_pca = pca.transform(test_audio_scaled)

print(f" Nombre de composantes retenues : {pca.n_components_}")

# --- Fusion des composantes principales (audio) avec les autres variables ---
X_final = np.concatenate([X_pca, X_other.values], axis=1)
test_final = np.concatenate([test_pca, test_other.values], axis=1)

print(f"Dimensions : train = {X_final.shape}, test = {test_final.shape}")

# --- Split apprentissage/validation (80/20) ---
X_train, X_val, y_train, y_val = train_test_split(X_final, y, test_size=0.2, random_state=42)

# --- Modèle de Régression Linéaire (moindres carrés ordinaires) ---
reg = LinearRegression()
reg.fit(X_train, y_train)

# --- Évaluation sur le set de validation ---
y_pred = reg.predict(X_val)
r2_val = r2_score(y_val, y_pred)
print(f" R² sur le jeu de validation : {r2_val:.4f}")

# --- Prédictions sur le jeu de test et création du fichier de soumission ---
preds = reg.predict(test_final)

submission = pd.DataFrame({
    "row_id": test_ids,
    "popularity": preds
})

submission.to_csv("submission_linear_pca.csv", index=False, sep=",", encoding="utf-8", float_format="%.6f")
print("Fichier 'submission_linear_pca.csv' créé avec succès !")


 Nombre de composantes retenues : 7
Dimensions : train = (85500, 126), test = (28500, 126)
 R² sur le jeu de validation : 0.2614
Fichier 'submission_linear_pca.csv' créé avec succès !


#  Random forest

In [ ]:
# ------------------------------------------------------------
#  Fonction utilitaire : conversion robuste des données en numérique
# ------------------------------------------------------------

def to_numeric_df(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in out.columns:
        out[c] = pd.to_numeric(out[c].astype(str).str.replace(',', '.', regex=False), errors='coerce')
    return out.fillna(0)

In [ ]:

# ------------------------------------------------------------
#  Chargement des données
# ------------------------------------------------------------
train = pd.read_csv("train_data_with_genres.csv", sep=";", encoding="utf-8", low_memory=False)
test = pd.read_csv("test_data_with_genres.csv", sep=";", encoding="utf-8", low_memory=False)

# Extraction de la variable cible

y = train["popularity"].values

# Sauvegarde des row_id pour la soumission finale

test_row_id = test["row_id"].values if "row_id" in test.columns else np.arange(len(test))

# Suppression des colonnes non nécessaires pour l'entraînement

drop_cols = [c for c in ["row_id", "popularity"] if c in train.columns]
train = train.drop(columns=drop_cols, errors="ignore")
test = test.drop(columns=["row_id"], errors="ignore")

# Conversion en numérique (corrige "0,754", "5,00E-05", etc.)

train = to_numeric_df(train)
test = to_numeric_df(test)

In [ ]:
# ------------------------------------------------------------
#  PCA (Analyse en Composantes Principales) sur les variables audio
# ------------------------------------------------------------
# Liste des colonnes audio sur lesquelles appliquer la PCA

audio_features = [
    "danceability", "energy", "loudness", "speechiness", "acousticness",
    "instrumentalness", "liveness", "valence", "tempo"
]
# On garde seulement celles présentes dans les données (sécurité)

audio_features = [f for f in audio_features if f in train.columns]
# Application de la PCA : 90 % de variance expliquée

pca = PCA(n_components=0.90, svd_solver="full", random_state=42)
X_audio_train = train[audio_features].values
X_audio_test = test[audio_features].values

# Transformation PCA

X_pca_train = pca.fit_transform(X_audio_train)

X_pca_test = pca.transform(X_audio_test)

# Création de DataFrames pour les composantes principales

pc_cols = [f"PC{i+1}" for i in range(X_pca_train.shape[1])]
df_pca_train = pd.DataFrame(X_pca_train, columns=pc_cols, index=train.index)
df_pca_test = pd.DataFrame(X_pca_test, columns=pc_cols, index=test.index)
# On conserve aussi les autres features (non audio)

X_base_train = train.drop(columns=audio_features, errors="ignore")
X_base_test = test.drop(columns=audio_features, errors="ignore")
# Fusion des features PCA + autres

X_full_train = pd.concat([df_pca_train, X_base_train], axis=1)
X_full_test = pd.concat([df_pca_test, X_base_test], axis=1)
# Alignement des colonnes train/test (important avant prédiction)

X_full_test = X_full_test.reindex(columns=X_full_train.columns, fill_value=0)

In [ ]:
# ------------------------------------------------------------
#  Division du jeu de données (80% train / 20% validation)
# ------------------------------------------------------------
X_train, X_val, y_train, y_val = train_test_split(X_full_train, y, test_size=0.2, random_state=42)

# ------------------------------------------------------------
#  Optimisation des hyperparamètres
# ------------------------------------------------------------
param_distributions = {
    'n_estimators': [300,400,500, 600, 700, 800, 900, 1000],
    'max_features': [0.3, 0.4, 0.5, 0.6 ,0.7,0.8, 'sqrt', 'log2'],
    'max_depth': [None, 20, 30, 40, 50, 60,70,80],  
    'min_samples_split': [2,3,4, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6],
    'bootstrap': [True]
}

# Modèle de base avec score OOB activé (Out-of-Bag)

rf_base = RandomForestRegressor(oob_score=True, n_jobs=-1, random_state=42)

# --- Étape 1 : RandomizedSearchCV (exploration large de l’espace) ---

random_search = RandomizedSearchCV(
    estimator=rf_base,
    param_distributions=param_distributions,
    n_iter=30,
    cv=3,
    scoring='r2',
    n_jobs=-1,
    verbose=0,
    random_state=42
)
random_search.fit(X_train, y_train)

best_params = random_search.best_params_

# --- Étape 2 : GridSearchCV (affinage local autour des meilleurs params) ---

def create_neighborhood(value, param_name):
    if param_name == 'n_estimators':
        return [max(100, value - 100), value, value + 100]
    elif param_name == 'max_depth':
        return [max(10, value - 10), value, value + 10, None] if value else [None, 40, 50, 60]
    elif param_name == 'min_samples_split':
        return [max(2, value - 2), value, value + 2]
    elif param_name == 'min_samples_leaf':
        return [max(1, value - 1), value, value + 1]
    elif param_name == 'max_features':
        return [value] if isinstance(value, str) else [max(0.2, value - 0.1), value, min(1.0, value + 0.1)]
    return [value]

param_grid = {p: create_neighborhood(v, p) for p, v in best_params.items()}

# Lancement du GridSearch (plus précis mais local)

grid_search = GridSearchCV(
    estimator=rf_base,
    param_grid=param_grid,
    cv=3,
    scoring='r2',
    n_jobs=-1,
    verbose=0
)
grid_search.fit(X_train, y_train)

# Meilleur modèle trouvé après les deux recherches

rf = grid_search.best_estimator_
best_params_final = grid_search.best_params_

In [ ]:

# ------------------------------------------------------------
#  Évaluation des performances sur le hold-out
# ------------------------------------------------------------
r2_tr = r2_score(y_train, rf.predict(X_train))
r2_val = r2_score(y_val, rf.predict(X_val))
print(f"R² (train): {r2_tr:.4f} | R² (val): {r2_val:.4f}")

# Calcul des importances de features via permutation_importance

try:
    perm = permutation_importance(rf, X_val, y_val, n_repeats=5, random_state=42, n_jobs=-1)
    imp = (
        pd.DataFrame({"feature": X_val.columns, "perm_importance": perm.importances_mean})
        .sort_values("perm_importance", ascending=False)
        .head(10)
    )
    print("\nTop 10 features importantes :")
    print(imp.to_string(index=False))
except Exception:
    pass


In [ ]:
# ------------------------------------------------------------
#  Entraînement final sur tout le jeu d'entraînement + prédictions test
# ------------------------------------------------------------
rf_final = RandomForestRegressor(**best_params_final, oob_score=False, n_jobs=-1, random_state=42)
rf_final.fit(X_full_train, y)
preds_test = rf_final.predict(X_full_test)

submission = pd.DataFrame({"row_id": test_row_id, "popularity": preds_test})
submission.to_csv("submission_random_forest.csv", index=False, sep=",", encoding="utf-8")

In [ ]:
# ------------------------------------------------------------
#  Sauvegarde des hyperparamètres et résultats
# ------------------------------------------------------------
with open("best_hyperparameters.txt", "w") as f:
    f.write("Optimisation des hyperparamètres Random Forest\n")
    f.write("RandomizedSearchCV:\n")
    for param, value in random_search.best_params_.items():
        f.write(f"{param}: {value}\n")
    f.write(f"Best CV R²: {random_search.best_score_:.4f}\n\n")
    f.write("GridSearchCV:\n")
    for param, value in best_params_final.items():
        f.write(f"{param}: {value}\n")
    f.write(f"Best CV R²: {grid_search.best_score_:.4f}\n")
    f.write(f"Validation R²: {r2_val:.4f}\n")


#   CATBOOST REGRESSOR

In [ ]:
# --- Chargement des données ---

print("48+ Chargement des données...")
train = pd.read_csv("train_data_with_genres.csv", sep=";", encoding="utf-8", low_memory=False)
test = pd.read_csv("test_data_with_genres.csv", sep=";", encoding="utf-8", low_memory=False)

# Vérification de la présence de la colonne cible 'popularity'

if "popularity" not in train.columns:
    raise ValueError("La colonne 'popularity' est absente du fichier d'entraînement.")
# Définition de la variable cible (y) et des features (X)

y_train = train["popularity"]
X_train = train.drop(columns=["popularity"], errors="ignore")

# Suppression éventuelle de la colonne 'row_id' dans le jeu d'entraînement

if "row_id" in X_train.columns:
    X_train = X_train.drop(columns=["row_id"])
# Même nettoyage pour le jeu de test

if "row_id" in test.columns:
    row_ids = test["row_id"]
    test = test.drop(columns=["row_id"])
else:
# Si la colonne n'existe pas, on crée un identifiant par défaut
    row_ids = pd.Series(range(len(test)))


In [ ]:
# Conversion de toutes les colonnes en valeurs numériques
# Les valeurs non convertibles deviennent NaN
X_train = X_train.apply(pd.to_numeric, errors="coerce")
test = test.apply(pd.to_numeric, errors="coerce")
# Remplacement des valeurs manquantes par la médiane de chaque colonne

X_train = X_train.fillna(X_train.median())
test = test.fillna(test.median())
# Affichage des dimensions du dataset préparé

print(f" Données préparées : {X_train.shape[0]} échantillons, {X_train.shape[1]} features")
# --- Définition de la validation croisée ---

N_FOLDS = 5
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
print("\n Recherche des meilleurs hyperparamètres avec Random Search...")

# --- Définition de l'espace de recherche des hyperparamètres ---

param_distributions = {
    'iterations': randint(400, 10000),
    'learning_rate': uniform(0.01, 0.1),  
    'depth': randint(6, 11),  
    'l2_leaf_reg': uniform(1, 9),  
    'bagging_temperature': uniform(0.1, 1), 
    'border_count': [64, 128, 254]
}

# --- Définition du modèle de base ---

base_model = CatBoostRegressor(
    random_seed=42,
    boosting_type='Plain',
    loss_function='RMSE',
    verbose=False
)
# --- Recherche aléatoire d'hyperparamètres ---

random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_distributions,
    n_iter=30,  
    cv=kf,
    scoring='r2',
    n_jobs=-1,  
    random_state=42,
    verbose=2
)
# Entraînement du modèle sur le train avec recherche aléatoire

random_search.fit(X_train, y_train)

In [ ]:
# --- Résultats de la recherche ---

print("\n Meilleurs hyperparamètres trouvés :")
best_params = random_search.best_params_
for param, value in best_params.items():
    print(f"  • {param}: {value}")
print(f"\n Meilleur R² moyen en CV : {random_search.best_score_:.4f}")

# Conversion des résultats de la recherche en DataFrame pour analyse
results_df = pd.DataFrame(random_search.cv_results_)
# Affichage des 5 meilleures combinaisons

print(f"\n Top 5 configurations :")
top5 = results_df.nlargest(5, 'mean_test_score')[['mean_test_score', 'std_test_score', 'params']]
for idx, row in top5.iterrows():
    print(f"  R² = {row['mean_test_score']:.4f} ± {row['std_test_score']:.4f}")

# --- Validation croisée manuelle avec les meilleurs paramètres ---

print("\n Entraînement du modèle final avec validation croisée...")

# Mise à jour des paramètres finaux du modèle

best_params['random_seed'] = 42
best_params['boosting_type'] = 'Plain'
best_params['loss_function'] = 'RMSE'
best_params['eval_metric'] = 'R2'
best_params['verbose'] = 200

# Listes pour stocker les scores et modèles

cv_r2_scores = []
cv_rmse_scores = []
models = []


In [ ]:
# Boucle sur les 5 folds

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
    print(f"\n--- Fold {fold + 1}/{N_FOLDS} ---")
    # Séparation des données train/validation pour le fold courant

    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

    # Création et entraînement du modèle

    model = CatBoostRegressor(**best_params)
    model.fit(X_tr, y_tr, eval_set=(X_val, y_val), early_stopping_rounds=100)
    
    # Prédictions sur le set de validation

    preds_val = model.predict(X_val)

    # Calcul des métriques de performance

    r2 = r2_score(y_val, preds_val)
    rmse = np.sqrt(mean_squared_error(y_val, preds_val))

    # Sauvegarde des scores et du modèle

    cv_r2_scores.append(r2)
    cv_rmse_scores.append(rmse)
    models.append(model)
    
    print(f"  R² : {r2:.4f} | RMSE : {rmse:.4f}")

In [ ]:
# Affichage des scores moyens sur les folds

print(f"\n Résultats de la validation croisée ({N_FOLDS} folds) :")
print(f"  • R² moyen  : {np.mean(cv_r2_scores):.4f} ± {np.std(cv_r2_scores):.4f}")
print(f"  • RMSE moyen: {np.mean(cv_rmse_scores):.4f} ± {np.std(cv_rmse_scores):.4f}")
# --- Entraînement final du modèle sur toutes les données ---

print("\n Entraînement du modèle final sur toutes les données d'entraînement...")
final_model = CatBoostRegressor(**best_params)
final_model.fit(X_train, y_train)
# Évaluation sur le train complet


y_pred_train = final_model.predict(X_train)
r2_train = r2_score(y_train, y_pred_train)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
print(f"  R² sur train complet  : {r2_train:.4f}")
print(f"  RMSE sur train complet: {rmse_train:.4f}")

# Sauvegarde du modèle et des hyperparamètres

joblib.dump(final_model, "catboost_spotify_model_optimized.pkl")
print("\n Modèle sauvegardé sous 'catboost_spotify_model_optimized.pkl'")

joblib.dump(best_params, "best_hyperparameters.pkl")
print(" Hyperparamètres sauvegardés sous 'best_hyperparameters.pkl'")

# --- Prédictions sur le jeu de test ---

print("\n Prédictions sur le jeu de test...")
# Moyenne des prédictions des 5 modèles de CV (ensemble)

test_preds = np.zeros(len(test))
for model in models:
    test_preds += model.predict(test) / len(models)

# Création du fichier de soumission

submission = pd.DataFrame({
    "row_id": row_ids,
    "popularity": test_preds
})
submission.to_csv("submission_catboost_optimized.csv", index=False)
print(" Fichier 'submission_catboost_optimized.csv' créé avec succès !")

In [ ]:
# --- Vérification des résultats ---

print("\n Aperçu de la soumission :")
print(submission.head(10))
# Statistiques descriptives des prédictions

print(f"\nStatistiques des prédictions :")
print(f"  • Min  : {test_preds.min():.2f}")
print(f"  • Max  : {test_preds.max():.2f}")
print(f"  • Moyen: {test_preds.mean():.2f}")
print(f"  • Std  : {test_preds.std():.2f}")
# --- Importance des features ---
print("\n Top 10 features importantes :")
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)
print(feature_importance.head(10).to_string(index=False))